# 🎮 Steam ACF Generator for Google Colab

**Generate Steam App Configuration Files (ACF) easily!**

This tool uses SKSAppManifestGenerator to create ACF files for your Steam App IDs.

## 📋 How to Use:
1. **Install Wine** (if needed) - Run the cell below
2. **Install Dependencies** - Run the next cell  
3. **Configure & Generate** - Use the interactive form below

## ⚠️ Important Notes:
- **Windows Tool**: SKSAppManifestGenerator is a Windows executable (.exe)
- **Colab Limitation**: Google Colab runs on Linux, so Wine may be needed
- **Best Option**: For best results, use the PowerShell script on Windows

## 🙏 Credits
- Original tool: **SKSAppManifestGenerator** by [Sak32009](https://github.com/Sak32009/SKSAppManifestGenerator)
- This wrapper provides a Python interface for Colab users


In [ ]:
# Install Wine if needed (for Linux/Colab to run Windows executables)
# Uncomment the line below if Wine is not available
# !apt-get update && apt-get install -y wine


In [ ]:
!pip install requests ipywidgets -q


In [ ]:
#@title Steam ACF Generator UI (Form)
# Hidden code - Setup and functions
import os
import sys
import requests
import zipfile
import subprocess
import shutil
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import unicodedata
import platform

# Configuration
PRIMARY_URL = 'https://github.com/Sak32009/SKSAppManifestGenerator/releases/download/v2.0.3/SKSAppManifestGenerator_x64_v2.0.3.zip'
FALLBACK_URL = 'https://github.com/ahmed98Osama/Steam-acf-generator/raw/master/SKSAppManifestGenerator_x64.exe'
TOOL_DIR = './tools/SKSAppManifestGenerator'
TOOL_NAME = 'SKSAppManifestGenerator_x64.exe'
TOOL_PATH = os.path.join(TOOL_DIR, TOOL_NAME)
ZIP_PASSWORD = b'cs.rin.ru'

# Colors for output
class Colors:
    CYAN = '\033[96m'
    YELLOW = '\033[93m'
    GREEN = '\033[92m'
    RED = '\033[91m'
    GRAY = '\033[90m'
    RESET = '\033[0m'

def print_info(msg):
    print(f"{Colors.CYAN}[INFO]{Colors.RESET} {msg}")

def print_warn(msg):
    print(f"{Colors.YELLOW}[WARN]{Colors.RESET} {msg}")

def print_err(msg):
    print(f"{Colors.RED}[ERROR]{Colors.RESET} {msg}")

def print_success(msg):
    print(f"{Colors.GREEN}[SUCCESS]{Colors.RESET} {msg}")

def _progress_bar(pct: float, width: int = 30) -> str:
    pct = max(0.0, min(100.0, pct))
    filled = int(round((pct / 100.0) * width))
    return '[' + ('#' * filled) + ('-' * (width - filled)) + ']'

def download_file(url, output_path):
    """Download a file from URL (fast + reliable: wget first, then curl, then Python with a single widget bar)."""
    try:
        print_info(f"Downloading from: {url}")
        # 1) Prefer wget for speed/reliability when available
        wget = shutil.which('wget') or shutil.which('wget.exe')
        if wget:
            tmp_out = output_path + '.part'
            args = [
                '--tries=3', '--timeout=20', '--read-timeout=20', '--no-verbose',
                '-O', tmp_out, url
            ]
            proc = subprocess.run([wget] + args, capture_output=True, text=True)
            if proc.returncode == 0 and os.path.exists(tmp_out) and os.path.getsize(tmp_out) > 0:
                os.replace(tmp_out, output_path)
                return True
            else:
                if os.path.exists(tmp_out):
                    try: os.remove(tmp_out)
                    except: pass
                print_warn(f"wget failed (code {proc.returncode}); trying curl...")
        
        # 2) Try curl next (Colab usually has it)
        curl = shutil.which('curl') or shutil.which('curl.exe')
        if curl:
            tmp_out = output_path + '.part'
            args = [
                '-fL', '--retry', '3', '--retry-delay', '2',
                '--connect-timeout', '20', '--max-time', '600',
                '-A', 'Colab-ACFDownloader/1.0',
                '-o', tmp_out, url
            ]
            proc = subprocess.run([curl] + args, capture_output=True, text=True)
            if proc.returncode == 0 and os.path.exists(tmp_out) and os.path.getsize(tmp_out) > 0:
                os.replace(tmp_out, output_path)
                return True
            else:
                if os.path.exists(tmp_out):
                    try: os.remove(tmp_out)
                    except: pass
                print_warn(f"curl failed (code {proc.returncode}); falling back to Python HTTP.")
        
        # 3) Fallback: requests streaming with a single widget progress bar
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        downloaded = 0
        
        prog = widgets.IntProgress(value=0, min=0, max=100, description='Downloading', layout=widgets.Layout(width='50%'))
        prog.style.bar_color = '#22c55e'
        label = widgets.HTML(value='0.0%')
        box = widgets.HBox([prog, label])
        display(box)
        
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    f.write(chunk)
                    downloaded += len(chunk)
                    if total_size > 0:
                        percent = (downloaded / total_size) * 100
                        prog.value = int(percent)
                        label.value = f"{percent:.1f}%"
        if total_size > 0:
            prog.value = 100
            label.value = '100%'
        return True
    except Exception as e:
        print_warn(f"Download failed: {str(e)}")
        return False

def extract_zip(zip_path, extract_to, password: bytes | None = None):
    """Extract ZIP file"""
    try:
        print_info(f"Extracting {zip_path} to {extract_to}")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            if password:
                try:
                    zip_ref.extractall(extract_to, pwd=password)
                except RuntimeError as re:
                    print_warn(f"Passworded extraction failed ({re}); retrying without password...")
                    zip_ref.extractall(extract_to)
            else:
                zip_ref.extractall(extract_to)
        return True
    except Exception as e:
        print_warn(f"Extraction failed: {str(e)}")
        return False

def find_executable(directory):
    """Find the executable in extracted directory"""
    for root, dirs, files in os.walk(directory):
        if TOOL_NAME in files:
            return os.path.join(root, TOOL_NAME)
    return None

def setup_tool():
    """Download and setup SKSAppManifestGenerator"""
    if os.path.exists(TOOL_PATH):
        print_info(f"Tool found at: {TOOL_PATH}")
        return TOOL_PATH
    
    print_warn(f"Tool not found at: {TOOL_PATH}")
    os.makedirs(TOOL_DIR, exist_ok=True)
    
    temp_zip = '/tmp/sks_generator.zip'
    temp_extract = '/tmp/sks_generator_extract'
    
    print_info("Attempting download from primary source...")
    if download_file(PRIMARY_URL, temp_zip):
        os.makedirs(temp_extract, exist_ok=True)
        if extract_zip(temp_zip, temp_extract, password=ZIP_PASSWORD):
            exe_path = find_executable(temp_extract)
            if exe_path:
                shutil.copy2(exe_path, TOOL_PATH)
                os.chmod(TOOL_PATH, 0o755)
                os.remove(temp_zip)
                shutil.rmtree(temp_extract, ignore_errors=True)
                print_success(f"Tool installed at: {TOOL_PATH}")
                return TOOL_PATH
    
    print_info("Primary download failed. Attempting fallback source...")
    if download_file(FALLBACK_URL, TOOL_PATH):
        os.chmod(TOOL_PATH, 0o755)
        print_success(f"Tool installed from fallback at: {TOOL_PATH}")
        return TOOL_PATH
    
    print_err("Failed to download tool from both sources")
    return None


def ensure_wine() -> str | None:
    """Ensure Wine is available on Linux/Colab. Returns the wine command to use or None."""
    system = platform.system().lower()
    if 'windows' in system:
        return None
    # check existing wine binaries
    for candidate in ['wine', 'wine64']:
        path = shutil.which(candidate)
        if path:
            return candidate
    # Attempt automatic install on Debian/Ubuntu environments (e.g., Colab)
    print_info("Wine not detected. Attempting to install Wine...")
    apt = shutil.which('apt-get')
    sudo = shutil.which('sudo')
    try:
        cmds = []
        if apt:
            cmds.append([apt, 'update'])
            # Prefer wine64; some environments only provide 'wine'
            cmds.append([apt, 'install', '-y', 'wine', 'wine64'])
        for cmd in cmds:
            print_info(f"Running: {' '.join(cmd)}")
            subprocess.run(cmd, check=True)
    except Exception as e:
        print_warn(f"Automatic Wine installation failed: {e}")
    # Re-check
    for candidate in ['wine', 'wine64']:
        path = shutil.which(candidate)
        if path:
            print_success(f"Wine installed: {candidate}")
            return candidate
    print_err("Wine is not available. Please install Wine and retry.")
    return None

def validate_app_ids(app_ids_input):
    """Validate and normalize App IDs (supports non-ASCII digits and mixed separators)."""
    if not app_ids_input or not app_ids_input.strip():
        return []

    def convert_to_ascii_digits(text: str) -> str:
        if not text:
            return text
        out = []
        for ch in text:
            try:
                dec = unicodedata.decimal(ch)
                out.append(chr(ord('0') + int(dec)))
            except Exception:
                out.append(ch)
        return ''.join(out)

    def extract_digits_only(text: str):
        if not text:
            return []
        text = convert_to_ascii_digits(text)
        tokens, current = [], []
        for ch in text:
            if '0' <= ch <= '9':
                current.append(ch)
            elif current:
                tokens.append(''.join(current))
                current = []
        if current:
            tokens.append(''.join(current))
        return tokens

    return extract_digits_only(app_ids_input)

def generate_acf_files(tool_path, app_ids, debug=False, working_dir=None):
    """Generate ACF files for given App IDs"""
    if working_dir is None:
        working_dir = os.getcwd()
    
    os.makedirs(working_dir, exist_ok=True)
    
    cmd = [tool_path]
    if debug:
        cmd.append('-d')
    cmd.extend(app_ids)
    
    print_info(f"Generating ACF files for App IDs: {', '.join(app_ids)}")
    print_info(f"Working directory: {working_dir}")
    
    original_dir = os.getcwd()
    try:
        os.chdir(working_dir)
        
        print_info("Attempting to execute generator...")
        wine_cmd = None
        if platform.system().lower() != 'windows':
            # Ensure Wine on non-Windows systems
            wine_cmd = ensure_wine()
        
        if wine_cmd:
            print_info(f"Using {wine_cmd} to run Windows executable.")
            cmd = [wine_cmd] + cmd
        
        try:
            result = subprocess.run(cmd, capture_output=True, text=False, timeout=600)
            stdout_text = (result.stdout or b'').decode('utf-8', errors='replace')
            stderr_text = (result.stderr or b'').decode('utf-8', errors='replace')
            if result.returncode == 0:
                print_success("ACF files generated successfully!")
                print(stdout_text)
            else:
                print_warn(f"Generator returned code {result.returncode}")
                print(stderr_text)
        except FileNotFoundError:
            print_err("Cannot execute Windows executable on Linux.")
            print_info("Please use the PowerShell script on Windows, or install Wine.")
        except OSError as e:
            # Retry with Wine if we hit Exec format error and haven't used wine yet
            msg = str(e)
            if ('Exec format error' in msg or 'format error' in msg) and (not wine_cmd) and platform.system().lower() != 'windows':
                print_warn("Exec format error detected. Retrying with Wine...")
                wine_cmd_retry = ensure_wine()
                if wine_cmd_retry:
                    try:
                        result = subprocess.run([wine_cmd_retry] + cmd, capture_output=True, text=False, timeout=600)
                        stdout_text = (result.stdout or b'').decode('utf-8', errors='replace')
                        stderr_text = (result.stderr or b'').decode('utf-8', errors='replace')
                        if result.returncode == 0:
                            print_success("ACF files generated successfully!")
                            print(stdout_text)
                        else:
                            print_warn(f"Generator returned code {result.returncode}")
                            print(stderr_text)
                    except Exception as e2:
                        print_err(f"Retry with Wine failed: {e2}")
                else:
                    print_err("Wine not available; cannot run Windows executable on Linux.")
            else:
                raise
        except subprocess.TimeoutExpired:
            print_err("Generator timed out")
    
    finally:
        os.chdir(original_dir)
    
    found_files = []
    for app_id in app_ids:
        candidates = [
            os.path.join(working_dir, f"appmanifest_{app_id}.acf"),
            os.path.join(working_dir, f"{app_id}.acf")
        ]
        for candidate in candidates:
            if os.path.exists(candidate):
                found_files.append(candidate)
                break
    
    if found_files:
        print_success(f"Found {len(found_files)} generated file(s):")
        for f in found_files:
            print(f"  - {f}")
    else:
        print_warn("No ACF files detected. Check generator output above.")

# Global variables for widgets
tool_path = None
status_output = widgets.Output()

def on_generate_clicked(b):
    """Handle generate button click"""
    with status_output:
        clear_output()
        
        # Get values from widgets
        app_ids_input = app_id_input.value
        debug = debug_checkbox.value
        working_dir = working_dir_input.value if working_dir_input.value else os.getcwd()
        
        # Validate App IDs
        app_ids = validate_app_ids(app_ids_input)
        if not app_ids:
            print_err("No valid App IDs provided.")
            return
        
        print(f"{Colors.GREEN}Valid App IDs:{Colors.RESET} {', '.join(app_ids)}")
        
        # Setup tool if needed
        global tool_path
        if not tool_path:
            print_info("Setting up SKSAppManifestGenerator...")
            tool_path = setup_tool()
            if not tool_path:
                print_err("Could not setup SKSAppManifestGenerator. Exiting.")
                return
        
        # Generate files
        print("\n" + "=" * 50)
        generate_acf_files(tool_path, app_ids, debug, working_dir)
        
        print("\n" + "=" * 50)
        print_success("Process completed!")
        print("=" * 50 + "\n")

# Create widgets
app_id_input = widgets.Text(
    value='',
    placeholder='Enter App IDs separated by commas or spaces (e.g., 570, 730, 440)',
    description='App IDs:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)

debug_checkbox = widgets.Checkbox(
    value=False,
    description='Enable debug output',
    style={'description_width': 'initial'}
)

working_dir_input = widgets.Text(
    value='',
    placeholder=f'Leave empty to use current directory: {os.getcwd()}',
    description='Working Directory:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)

generate_button = widgets.Button(
    description='🚀 Generate ACF Files',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

generate_button.on_click(on_generate_clicked)

# Display the interface
display(HTML("""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            padding: 20px; border-radius: 10px; margin: 10px 0;">
    <h2 style="color: white; text-align: center; margin: 0;">🎮 Steam ACF Generator</h2>
    <p style="color: white; text-align: center; margin: 10px 0;">Generate Steam App Configuration Files easily!</p>
</div>
"""))

display(HTML("<h3>📝 Configuration</h3>"))
display(app_id_input)
display(debug_checkbox)
display(working_dir_input)

display(HTML("<br>"))
display(generate_button)

display(HTML("<h3>📊 Status & Output</h3>"))
display(status_output)
